## Visual QC notebook

This notebook is designed to create outputs used in a visual QC check of computed indicators. This notebook should be run as a final step of the Prefect indicator flow. The output will be saved as a new notebook. 

Create a task in the Prefect flow that will execute this notebook from the command line using `papermill`, e.g.:

```papermill path/to/repo/indicators/visual_qc.ipynb path/to/qc/output/output.ipynb -r indicators "rx1day" -r models "GFDL-ESM4 CESM2 TaiESM1" -r "historical ssp126 ssp245 ssp370 ssp585"```


 The first argument is this notebook's location, which can be constructed using the output parameter of the Prefect flow (ie, the notebook's location within the downloaded repo directory). The second argument is the desired notebook output location, which can be constructed using the output parameter of the Prefect flow. The remaining arguments for indicators, models, and scenarios should be copied directly from the Prefect flow input parameters. (Note that each parameter is preceded by the `-r` tag to designate a raw string, and the multiple items are enclosed in quotes and separated by a single white space.)






In [8]:
# this cell is tagged "parameters" and contains default values for this notebook
# these values are read when executing the notebook using papermill
# parameters injected from the command line that differ from these defaults will override the values in this cell
# use a raw string surrounded by quotes with values separated by a single blank space; this matches the raw string parameter input from the command line

indicators = "rx1day"
models = "GFDL-ESM4 CESM2 TaiESM1"
scenarios = "historical ssp126 ssp245 ssp370 ssp585"


In [9]:
indicators_, models_, scenarios_ = indicators.split(" "), models.split(" "), scenarios.split(" ")

print(indicators_)
print(models_)
print(scenarios_)

['rx1day']
['GFDL-ESM4', 'CESM2', 'TaiESM1']
['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
